#Dependencies#

In [1]:
!pip install transformers --quiet
!pip install datasets transformers[SentencePiece] --quiet  

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 61 kB 369 kB/s 
     |████████████████████████████████| 895 kB 41.3 MB/s 
     |████████████████████████████████| 3.3 MB 43.0 MB/s 
     |████████████████████████████████| 596 kB 32.0 MB/s 
     |████████████████████████████████| 298 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 46.0 MB/s 
     |████████████████████████████████| 132 kB 52.0 MB/s 
     |████████████████████████████████| 1.1 MB 38.4 MB/s 
     |████████████████████████████████| 160 kB 52.4 MB/s 
     |████████████████████████████████| 271 kB 59.2 MB/s 
     |████████████████████████████████| 192 kB 49.2 MB/s 
     |████████████████████████████████| 1.2 MB 53.1 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'CS685-Project/s2s-decipherment-multilingual'
%cd /content/drive/My\Drive/$FOLDERNAME/code

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1d104twHY-SPOagFGjLaTFEOZGkx2WNJs/CS685-Project/s2s-decipherment-multilingual/code


In [3]:
import sys
sys.path.append('/content/drive/My Drive/{}/code'.format(FOLDERNAME))

%load_ext autoreload
%autoreload 2

In [4]:
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModel, AdamW, DataCollatorWithPadding, get_scheduler, logging
import pandas as pd
import torch
from torch import Tensor
from torch.optim import *
import torch.optim
from torch.utils.data import DataLoader
from torch.nn import *
from torch.nn.functional import one_hot as get_one_hot_enc
from torch.nn.functional import cross_entropy
from torch.nn import Transformer, TransformerEncoder, TransformerEncoderLayer, Embedding, TransformerDecoderLayer, TransformerDecoder, Transformer
from tqdm.auto import tqdm
import numpy as np
from os import listdir
from os.path import isfile, join
import math
from lxml.html import fromstring
import lxml.html as PARSER
import string
import random
from collections import Counter
from models import Deciphormer
from data_utils import create_master_data, get_split_dfs, get_idx_to_chr, get_chr_to_idx
from data import CipherDataset
from train_test import validation_loop, fit
import copy
import time

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
device

device(type='cuda')

###Create Master Data###
Run the second cell only if you're restarting the notebook after you've run create_master_data() once

In [5]:
df,df_test,_ = create_master_data('/content/drive/My Drive/{}/data'.format(FOLDERNAME))
df_train, df_val = get_split_dfs(df=df)

In [6]:
# #Get train, test and val DataFrames; they will be useful in error analysis
# df_train, df_val = get_split_dfs('/content/drive/My Drive/{}/master_data/{}.csv'.format(FOLDERNAME,mode))
# df_test = pd.read_csv('/content/drive/My Drive/{}/master_data/{}.csv'.format(FOLDERNAME,'test'))

###Prepare Data###

In [7]:
#Define some constants for training
mode = 'dev'
batch_size = 16
epochs = 5
learning_rate = 1e-3
chr_to_idx = get_chr_to_idx()
idx_to_chr = get_idx_to_chr()

#Create Train, Test, and Val Dataloaders

train_dataset = CipherDataset(df_train)
val_dataset = CipherDataset(df_val)
test_dataset = CipherDataset(df_test)
raw_datasets = DatasetDict({'train':train_dataset, 'validation':val_dataset, 'test':test_dataset})

train_dataloader = DataLoader(raw_datasets['train'], batch_size=batch_size)
val_dataloader = DataLoader(raw_datasets['validation'], batch_size=batch_size)
test_dataloader = DataLoader(raw_datasets['test'], batch_size=batch_size)

###Train Model###

In [9]:
np.random.seed(0)
start = time.time()
model = Deciphormer(ntoken=len(get_chr_to_idx()))
optimizer = Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()
model.to(device)
train_loss_list, validation_loss_list, best_model = fit(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs = epochs)
end = time.time()
print(start-end)

  0%|          | 0/30625 [00:00<?, ?it/s]

Training and validating model
------------------------- Epoch 1 -------------------------
Iteration 1/6125: | Loss = 3.40
Iteration 100/6125: | Loss = 0.10
Iteration 200/6125: | Loss = 0.03
Iteration 300/6125: | Loss = 0.01
Iteration 400/6125: | Loss = 0.01
Iteration 500/6125: | Loss = 0.01
Iteration 600/6125: | Loss = 0.00
Iteration 700/6125: | Loss = 0.00
Iteration 800/6125: | Loss = 0.00
Iteration 900/6125: | Loss = 0.00
Iteration 1000/6125: | Loss = 0.00
Iteration 1100/6125: | Loss = 0.00
Iteration 1200/6125: | Loss = 0.00
Iteration 1300/6125: | Loss = 0.00
Iteration 1400/6125: | Loss = 0.00
Iteration 1500/6125: | Loss = 0.00
Iteration 1600/6125: | Loss = 0.00
Iteration 1700/6125: | Loss = 0.00
Iteration 1800/6125: | Loss = 0.00
Iteration 1900/6125: | Loss = 0.00
Iteration 2000/6125: | Loss = 0.00
Iteration 2100/6125: | Loss = 0.00
Iteration 2200/6125: | Loss = 0.00
Iteration 2300/6125: | Loss = 0.00
Iteration 2400/6125: | Loss = 0.00
Iteration 2500/6125: | Loss = 0.00
Iteration 26

KeyboardInterrupt: ignored

###Error Analysis###

In [10]:
#One-Shot Prediction
model = Deciphormer(ntoken=len(get_chr_to_idx()))
checkpoint = torch.load('/content/drive/My Drive/{}/output/saved_model.pth'.format(FOLDERNAME))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
df_pred = df_val.copy()
_, SER, pred = validation_loop(model, loss_fn, val_dataloader)
df_pred['pred'] = pred
display(df_pred)
df_pred.to_csv('/content/drive/My Drive/{}/output/predictions_baseline_one_shot.csv'.format(FOLDERNAME))

,text,input_ids,labels,lang,pred
97988,t h e _ p r e s e n c e _ o f _ s e v e r a l ...,28 1 8 0 26 17 7 0 2 0 3 13 0 26 6 9 26 2 0 18...,28 19 7 4 26 15 17 4 18 4 13 2 4 26 14 5 26 18...,1,t h e _ p r e s e n c e _ o f _ s e v e r a l ...
97989,p o r _ i s s o _ s e _ i n d i g n o u _ o _ ...,28 8 0 5 26 3 1 1 0 26 1 2 26 3 6 7 3 9 6 0 10...,28 15 14 17 26 8 18 18 14 26 18 4 26 8 13 3 8 ...,13,p o r _ i s s o _ s e _ i n d i g n o u _ o _ ...
97990,h a _ a l l o t j a t _ t o t _ u n _ e x e r ...,28 11 3 26 3 8 8 0 1 12 3 1 26 1 0 1 26 13 9 2...,28 7 0 26 0 11 11 14 19 9 0 19 26 19 14 19 26 ...,0,h a _ a l l o t j a t _ t o t _ u n _ e x e r ...
97991,n e m e s i _ k o r o n a t _ a _ z s e b k e ...,28 1 0 13 0 2 4 26 5 7 9 7 1 3 8 26 3 26 10 2 ...,28 13 4 12 4 18 8 26 10 14 17 14 13 0 19 26 0 ...,7,n e m e s i _ k o r o n a t _ a _ z s e b k e ...
97992,d e r _ h v o r _ v a n d e t,28 0 1 2 26 4 3 5 2 26 3 6 7 0 1 8 29 27 27 27...,28 3 4 17 26 7 21 14 17 26 21 0 13 3 4 19 29 2...,8,d e r _ h v o r _ v a n d e t <EOS>
...,...,...,...,...,...
99983,j o u t u n u t _ k a m m o a m a a n s a _ p ...,28 10 4 3 2 3 1 3 2 26 5 0 7 7 4 0 7 0 0 1 8 0...,28 9 14 20 19 20 13 20 19 26 10 0 12 12 14 0 1...,6,j o u t u n u t _ k a m m o a m a a n s a _ p ...
99984,s a l o m o _ i n d e _ f o m e s _ i m p i e ...,28 1 5 9 3 4 3 26 2 10 7 0 26 13 3 4 0 1 26 2 ...,28 18 0 11 14 12 14 26 8 13 3 4 26 5 14 12 4 1...,3,s a l o m o _ i n d e _ f o m e s _ i m p i e ...
99985,b a h a g y a _ m a y _ k a h i r a p a n,28 3 0 1 0 4 2 0 26 5 0 2 26 6 0 1 7 8 0 9 0 1...,28 1 0 7 0 6 24 0 26 12 0 24 26 10 0 7 8 17 0 ...,4,b a h a g y a _ m a y _ k a h i r a p a n <EOS>
99986,j u g u e t e _ d e _ v o c a b l o s,28 3 1 4 1 0 5 0 26 6 0 26 7 2 8 9 10 11 2 12 ...,28 9 20 6 20 4 19 4 26 3 4 26 21 14 2 0 1 11 1...,4,j u g u e t e _ d e _ v o c a b l o s <EOS>
